## OpenAI Vision 4 Image-to-Text Descriptions of First Frames (focusing on text, i.e. color, size, placement)

In [ ]:
!pip install opencv-python
!pip install openai
!pip install pytube

In [ ]:
from IPython.display import display, Image, Audio

import cv2
import base64
import time
import os
import requests

from openai import OpenAI
client = OpenAI(api_key="")

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

## Loop through Training Set, create OpenAI descriptions

In [ ]:
import os
import base64
import pandas as pd
import numpy as np
import time
import requests
import json
from openai import BadRequestError, RateLimitError

# Function to convert image to base64
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Directory containing images
image_dir = "/content/drive/My Drive/236/train_first_frames/"

In [ ]:
# Load train.csv into a DataFrame
train_df = pd.read_csv("/content/drive/My Drive/236/train.csv")

print (len(train_df))
train_df["OpenAICaption"] = np.nan
train_df

In [ ]:
# Load test.csv, when testing
image_dir = "/content/drive/My Drive/236/test_first_frames/"
test_df = pd.read_csv("/content/drive/My Drive/236/test.csv")
test_df["OpenAICaption"] = np.nan

### Batch calls

In [ ]:
# TRAIN SET

id_set = set(train_df['Id'].values)
progress = 0
batch_size = 5
batch = []

for filename in os.listdir(image_dir):
    if filename.endswith(".jpg"):
        identifier = filename.split("_", 1)[1][:-4]  # Removes the ".jpg" at the end

        if identifier in id_set and not pd.isna(train_df.loc[train_df['Id'] == identifier, 'OpenAICaption'].iloc[0]):
            continue

        base64_image = encode_image(image_dir + filename)

        # Append to the batch
        batch.append({
            "image": base64_image,
            "identifier": identifier
        })


        if len(batch) == batch_size:
            message = {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "Each image is the first frame of a YouTube video. Generate an around 30-word description for each image that would be useful for creating a good thumbnail image, including subject matter, composition, style and descriptions of how any text should look, including text size, color, and placement. List each description on a new line without numbering. If you are unable to provide a description, return 'no description available'",
                    }
                ]
            }

            for item in batch:
                batch_image = {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{item['image']}"
                            }
                        }
                message["content"].append(batch_image)

            params = {
                "model": "gpt-4-vision-preview",
                "messages": [message],
                "max_tokens": 4000,
            }

            try:
              result = client.chat.completions.create(**params)
              result_list = result.choices[0].message.content.split("\n")

              for i in range(batch_size):
                description = result_list[i]
                identifier = batch[i]["identifier"]
                train_df.loc[train_df['Id'] == identifier, 'OpenAICaption'] = description
                print(f"{identifier}: {description}")

              progress += batch_size
              print(progress)

              train_df.to_csv("/content/drive/My Drive/train_images_exp4_openai.csv", index=False)

            except (RateLimitError) as e:
                print(f"Error: {e}, waiting...")
                time.sleep(90)
            except (BadRequestError) as e:
                print(f"Error: {e}, skipping batch.")
                batch = []
                continue
            except Exception as e:
                print(f"Unexpected error: {e}, skipping batch.")
                batch = []
                continue

            # Reset the batch
            batch = []

# Save the updated DataFrame back to CSV
train_df.to_csv("/content/drive/My Drive/train_images_exp4_openai.csv", index=False)


In [ ]:
#  TEST SET

id_set = set(test_df['Id'].values)
progress = 0
batch_size = 1
batch = []
num = 0

for filename in os.listdir(image_dir):
    num += 1
    if filename.endswith(".jpg"):
        identifier = filename.split("_", 1)[1][:-4]  # Removes the ".jpg" at the end

        if identifier in id_set and not pd.isna(test_df.loc[test_df['Id'] == identifier, 'OpenAICaption'].iloc[0]):
            num += 1
            continue

        base64_image = encode_image(image_dir + filename)

        # Append to the batch
        batch.append({
            "image": base64_image,
            "identifier": identifier
        })


        if len(batch) == batch_size:
            message = {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "Each image is the first frame of a YouTube video. Generate an around 30-word description for each image that would be useful for creating a good thumbnail image, including subject matter, composition, style and descriptions of how any text should look, including text size, color, and placement. List each description on a new line without numbering. If you are unable to provide a description, return 'no description available'",
                    }
                ]
            }

            for item in batch:
                batch_image = {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{item['image']}"
                            }
                        }
                message["content"].append(batch_image)

            params = {
                "model": "gpt-4-vision-preview",
                "messages": [message],
                "max_tokens": 4000
            }

            try:
              result = client.chat.completions.create(**params)
              result_list = result.choices[0].message.content.split("\n")

              for i in range(batch_size):
                description = result_list[i]
                identifier = batch[i]["identifier"]
                test_df.loc[test_df['Id'] == identifier, 'OpenAICaption'] = description
                print(f"{identifier}: {description}")

              progress += batch_size
              print(progress)

              test_df.to_csv("/content/drive/My Drive/236/test_images_exp4_openai.csv", index=False)
              time.sleep(30)

            except (RateLimitError) as e:
                print(f"Error: {e}, waiting...")
                time.sleep(90)
            except (BadRequestError) as e:
                print(f"Error: {e}, skipping batch.")
                batch = []
                continue
            except Exception as e:
                print(f"Unexpected error: {e}, skipping batch.")
                batch = []
                continue

            # Reset the batch
            batch = []

# Save the updated DataFrame back to CSV
test_df.to_csv("/content/drive/My Drive/236/test_images_exp4_openai.csv", index=False)
print (num)


### Individual calls

In [ ]:
import os
import base64
import pandas as pd
import time
from openai import BadRequestError, RateLimitError

# Function to convert image to base64
def image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# Directory containing images
image_dir = "/content/drive/My Drive/236/train_first_frames"
train_df = pd.read_csv("/content/drive/My Drive/train_images_exp2_openai.csv")

id_set = set(train_df['Id'].values)
progress = 0

for filename in os.listdir(image_dir):
    if filename.endswith(".jpg"):
        # Extract the identifier from the filename
        identifier = filename.split("_", 1)[1][:-4]  # Removes the ".jpg" at the

        if identifier in id_set and not pd.isna(train_df.loc[train_df['Id'] == identifier, 'OpenAICaption'].iloc[0]):
            continue

        image_path = os.path.join(image_dir, filename)
        base64_image = image_to_base64(image_path)

        # Create the prompt message
        prompt_message = {
            "role": "user",
            "content": [
                "This is the first frame from a YouTube video. Generate a less than 20 word description of the video that would be useful for creating a good thumbnail image.",
                {"image": base64_image, "resize": 256}
            ],
        }

        # Prepare parameters for the API call
        params = {
            "model": "gpt-4-vision-preview",
            "messages": [prompt_message],
            "max_tokens": 200
        }

        try:
            result = client.chat.completions.create(**params)
            description = result.choices[0].message.content

            if "I'm sorry" in description or "I cannot" in description:
                train_df.loc[train_df['Id'] == identifier, 'OpenAICaption'] = "no description available"
            else:
                train_df.loc[train_df['Id'] == identifier, 'OpenAICaption'] = description

            progress += 1
            print(progress)

            if progress % 10 == 0:  # Save every 10 iterations
                train_df.to_csv("/content/drive/My Drive/train_images_exp2_openai.csv", index=False)

            time.sleep(30)

        except (RateLimitError) as e:
            print(f"Error: {e}, waiting...")
            time.sleep(30)
        except (BadRequestError) as e:
            print(f"Error: {e}, skipping image.")
            train_df.loc[train_df['Id'] == identifier, 'OpenAICaption'] = "no description available"
            continue
        except Exception as e:
            print(f"Unexpected error: {e}, skipping image.")
            continue

# Save the updated DataFrame back to CSV
train_df.to_csv("/content/drive/My Drive/train_images_exp2_openai.csv", index=False)


### See Results

In [ ]:
# Save the updated DataFrame back to CSV
train_df.to_csv("/content/drive/My Drive/train_images_exp2_openai.csv", index=False)

In [ ]:
num = 0
for index, row in train_df.iterrows():
    if pd.notna(row['OpenAICaption']) and row['OpenAICaption'] != '':
        print(f"Row {index}: {row['OpenAICaption']}")
        num += 1
print (len(train_df))